In [2]:
import warnings
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

In [3]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [4]:
def new_column_names(df_copy):
    column_list = ['label']
    row, column = 1,1;
    while row < 29:
        while column < 29:
            column_name = str(row) + 'x' + str(column)
            column_list.append(column_name)
            column+=1
        row+=1;
        if(column == 29):
            column = 1
            
    df_copy.columns = column_list
    return df_copy

def split_dataset_to_train_test_validation(df_copy, train_sze, test_sze, validation_sze):
    random.seed(0)
    target_column = ['label']
    predictors = list(set(list(df.columns))-set(target_column))
    X = df_copy[predictors] #Contains all columns
    Y = df_copy[['label']]
    X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, train_size=train_sze, random_state=50)
    X_test, X_validation, Y_test, Y_validation = train_test_split(X_temp, Y_temp, train_size=test_sze/(test_sze+validation_sze), random_state=50)
    return X_train, Y_train, X_test, Y_test, X_validation, Y_validation

def normalizer(dataframe):
    result = dataframe.copy()/255
    return result

def model():
    inputs = Input(shape=(28,28,1))
    Conv2D(24,kernel_size=(3,3),padding='same',activation="relu", input_shape = (28,28,1))
    MaxPooling2D(pool_size=(2, 2))
    Conv2D(48, (3, 3), padding='same',activation='relu', input_shape = (28,28,1))
    MaxPooling2D(pool_size=(2, 2))
    Conv2D(64, (3, 3), padding='same',activation='relu', input_shape = (28,28,1))
    MaxPooling2D(pool_size=(2, 2))
    Flatten()
    Dense(128, activation='relu')
    Dropout(0.25)
    output = Dense(10,activation="softmax")
    
    model = Model(inputs,output)
    
    model.compile(loss='categorical_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])
    
    return model

In [5]:
warnings.filterwarnings('ignore')

In [6]:
df_train = pd.read_csv('MNIST_TRAIN.csv')
df_test = pd.read_csv('MNIST_TEST.csv')

In [7]:
print('Numbers of NaN values in training set: '); print(df_train.isnull().sum().sum()); 
print('Numbers of NaN values in test set: '); print(df_test.isnull().sum().sum());
print(df_train); print(df_test);

Numbers of NaN values in training set: 
0
Numbers of NaN values in test set: 
0
       label  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0          5    0    0    0    0    0    0    0    0    0  ...      0      0   
1          0    0    0    0    0    0    0    0    0    0  ...      0      0   
2          4    0    0    0    0    0    0    0    0    0  ...      0      0   
3          1    0    0    0    0    0    0    0    0    0  ...      0      0   
4          9    0    0    0    0    0    0    0    0    0  ...      0      0   
...      ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...    ...    ...   
59995      8    0    0    0    0    0    0    0    0    0  ...      0      0   
59996      3    0    0    0    0    0    0    0    0    0  ...      0      0   
59997      5    0    0    0    0    0    0    0    0    0  ...      0      0   
59998      6    0    0    0    0    0    0    0    0    0  ...      0      0   
59999      8    0    0    0    0    0   

In [8]:
X_train = df_train.drop('label', axis = 1)
Y_train = df_train[['label']]

X_test = df_test.drop('label', axis = 1)
Y_test = df_test[['label']]

X_train = normalizer(X_train); X_test = normalizer(X_test)

In [9]:
model = Sequential()
model.add(Dense(784, activation='sigmoid'))
model.add(Dense(16, activation='sigmoid'))
model.add(Dense(10))

2022-06-27 07:48:26.777400: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-06-27 07:48:26.794294: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: Tesla V100-PCIE-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:5e:00.0
2022-06-27 07:48:26.795488: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 1 with properties: 
name: Tesla V100-PCIE-16GB major: 7 minor: 0 memoryClockRate(GHz): 1.38
pciBusID: 0000:af:00.0
2022-06-27 07:48:26.795816: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.0
2022-06-27 07:48:26.797630: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcublas.so.10.0
2022-06-27 07:48:26.799290: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcufft.so.1

In [ ]:
model.fit(X_train.values, Y_train.values, epochs = 1)

In [ ]:
model.predict(X_test)

In [ ]:
model.summary()

In [ ]:
from platform import python_version

In [ ]:
python_version()

In [ ]:
!pip list

In [ ]:
print(tf.test.is_gpu_available())
print(tf.config.experimental.list_physical_devices('GPU'))

In [ ]:
tf.config.list_physical_devices('GPU')